In [1]:
import pandas as pd
import random
import math
from numpy.random import rand
import time

In [2]:
oversampling = True
train_part_num = 10
test_part_num = 1
k = 1
in_k = max(math.floor(k/train_part_num), 1)

In [3]:
if not oversampling:
    train_df = pd.read_csv("../data/ECG5000/ECG5000_TRAIN.tsv", sep='\t', header=None)
    test_df = pd.read_csv("../data/ECG5000/ECG5000_TEST.tsv" ,sep='\t', header=None)
    train_df = train_df.sample(frac=1, random_state=42)
    test_df = test_df.sample(frac=1, random_state=42)
    train_df = train_df.rename(columns={"0": "Class"})
    test_df = test_df.rename(columns={"0": "Class"})

    for col_name in train_df.columns[1:]:
        if col_name != "Class":
            train_df[col_name] = train_df[col_name].astype("float")

    for col_name in test_df.columns[1:]:
        if col_name != "Class":
            test_df[col_name] = test_df[col_name].astype("float")
    merged_df = pd.concat([train_df, test_df], ignore_index=True)

In [4]:
if oversampling:
    merged_df = pd.read_csv("../data/ECG5000/ECG5000_Oversampled.csv")
    merged_df = merged_df.sample(frac=1, random_state=42)
# Show the resulting dataframe
merged_df.shape[0]

6300

In [5]:
merged_df = merged_df.head(1000)

In [6]:
from sklearn.model_selection import train_test_split

train_ratio = 0.7
test_ratio = 0.3
random_state = 346

train_df, test_df = train_test_split(merged_df, train_size=train_ratio, test_size=test_ratio, random_state=random_state)


In [7]:
train_df.shape[0]

700

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Get X and y data
X_train = train_df.iloc[:, 1:].values
y_train = train_df["Class"].values
X_test = test_df.iloc[:, 1:].values
y_test = test_df["Class"].values

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)

start_time = time.time()

# Fit the classifier to the training data
knn.fit(X_train, y_train)
# Make predictions on the test data
y_pred = knn.predict(X_test)

predict_time = time.time() - start_time

print("Time to fit and make predictions:", predict_time)
# Print classification report
print(classification_report(y_test, y_pred))


Time to fit and make predictions: 0.22305989265441895
              precision    recall  f1-score   support

           1       0.98      0.96      0.97       130
           2       0.87      0.88      0.88        93
           3       0.77      0.77      0.77        22
           4       0.70      0.78      0.74        36
           5       0.94      0.84      0.89        19

    accuracy                           0.89       300
   macro avg       0.85      0.85      0.85       300
weighted avg       0.90      0.89      0.89       300

